In [14]:
import pandas as pd
messages=pd.read_csv(r'C:\Users\DELL\Videos\Email_Spam_Detection_Mchine_Learning-main\sms+spam+collection\SMSCollection.csv', sep='\t', names= ["label","message"])

import nltk
import re

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [17]:
messages.tail()

,label,message
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [27]:
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')


In [21]:
ps=PorterStemmer()
wnl=WordNetLemmatizer()
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['message'][i])
    review=review.lower()
    review=review.split()
    
    review=[wnl.lemmatize(word) for word in review if not word in satopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer #This line imports the CountVectorizer class from scikit-learn, which is used to convert a collection of text documents into a matrix of token counts.
cv=CountVectorizer(max_features=5000) #Here, you're creating a CountVectorizer object with a maximum of 5000 features. This means that only the top 5000 most frequent words will be considered as features.
X=cv.fit_transform(corpus).toarray() #This line fits the CountVectorizer to your corpus (presumably a list of text documents) and transforms the corpus into a document-term matrix represented as a NumPy array. Each row in this array corresponds to a document, and each column corresponds to a word in the vocabulary created by the CountVectorizer. The .toarray() method converts the sparse matrix returned by fit_transform into a dense NumPy array.

y=pd.get_dummies(messages['label']) #This line creates dummy variables for the 'label' column in your 'messages' DataFrame using pandas' get_dummies() function. Each unique label will become a separate column with binary values indicating the presence or absence of that label.
y=y.iloc[:,1].values #This line selects only one column from the dummy variable DataFrame y (presumably to avoid multicollinearity issues), converts it to a NumPy array, and assigns it to the variable y.
y

array([False, False,  True, ..., False, False, False])

In [23]:
#Train test split
from sklearn.model_selection import train_test_split #Imports the train_test_split function from the sklearn.model_selection module. This function is used to split arrays or matrices into random train and test subsets.
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.20,random_state=0) 
''' Splits the features (X) and target labels (y) into training and testing sets.
    X: The features (independent variables).
    y: The target labels (dependent variable).
    test_size=0.20: Specifies that 20% of the data will be used for testing, and the remaining 80% will be used for training.
    random_state=0: Sets the random seed for reproducibility. It ensures that each time you run the code, the same random splits are generated.'''
#Training model using Naive Bayes Classifier 
from sklearn.naive_bayes import MultinomialNB # Imports the MultinomialNB class from the sklearn.naive_bayes module. This class implements the Naive Bayes algorithm for multinomially distributed data.
mv=MultinomialNB() #Creates an instance of the MultinomialNB classifier.
spam_detection_model=mv.fit(X_train,y_train) #Fits the Naive Bayes classifier to the training data (X_train and y_train). The fit method trains the model on the provided training data.

y_pred=spam_detection_model.predict(X_test) #Uses the trained model (spam_detection_model) to make predictions on the test set (X_test). The predict method predicts the target labels based on the input features (X_test) and returns the predicted labels (y_pred), which can be compared with the actual labels (y_test) for evaluation.

In [24]:
y_pred

array([False,  True, False, ..., False,  True, False])

In [25]:
from sklearn.metrics import confusion_matrix
confusion_m=confusion_matrix(y_test,y_pred)

confusion_m

array([[944,  11],
       [  9, 151]], dtype=int64)

In [26]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)

accuracy

0.9820627802690582

In [29]:
import gradio as gr 

In [30]:
def predict_spam(message):
    processed_message = re.sub('[^a-zA-Z]', ' ', message)
    processed_message = processed_message.lower()
    processed_message = processed_message.split()
    processed_message = [wnl.lemmatize(word) for word in processed_message if not word in stopwords.words('english')]
    processed_message = ' '.join(processed_message)
    vectorized_message = cv.transform([processed_message]).toarray()
    prediction = spam_detection_model.predict(vectorized_message)[0]
    return "Spam" if prediction == 1 else "Not Spam"

# Create Gradio interface
interface = gr.Interface(fn=predict_spam, inputs="text", outputs="text", title="Spam Detection", description="Enter a message to check if it is spam or not.")

# Launch Gradio interface
interface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
